Датасет ml-latest.
* Вспомнить подходы, которые мы разбирали.
* Выбрать понравившийся подход к гибридным системам.
* Написать свою.

In [1]:
import pandas as pd

In [2]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-05-22 20:22:12--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.26.102, 74.125.26.100, 74.125.26.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/eodn11a7ul5f9b0mi1pg58oabchu0atr/1684786875000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=658506f8-3f67-48c9-8a3e-e9bbe2ac43a3 [following]
--2023-05-22 20:22:13--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/eodn11a7ul5f9b0mi1pg58oabchu0atr/1684786875000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=658506f8-3f67-48c9-8a3e-e9bbe2ac43a3
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to doc-0g-84-

In [3]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
dataset = ratings.drop('timestamp', axis = 1)

In [7]:
movies = pd.read_csv('movies.csv')

In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


**Обучаем первую модель гибридной рекомендательной системы,  используем алгоритм SVD++ (Singular Value Decomposition ++)**

In [9]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from surprise import SVDpp, Dataset, Reader
from surprise.model_selection import cross_validate

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [12]:
algo_SVDpp = SVDpp()

In [13]:
cross_validate(algo_SVDpp, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8588  0.8644  0.8572  0.8610  0.8607  0.8604  0.0024  
Fit time          110.48  98.12   96.73   98.44   96.06   99.97   5.33    
Test time         11.88   12.86   11.99   11.58   11.91   12.05   0.43    


{'test_rmse': array([0.85881371, 0.86438351, 0.85721042, 0.86102984, 0.86070107]),
 'fit_time': (110.48390626907349,
  98.11795377731323,
  96.72922348976135,
  98.44129276275635,
  96.05707287788391),
 'test_time': (11.883568048477173,
  12.85868215560913,
  11.99129319190979,
  11.583410024642944,
  11.90930438041687)}

**Обучаем вторую модель гибридной рекомендательной системы,  используем алгоритм KNNWithMeans**

In [14]:
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split

In [15]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [16]:
algo_KNN = KNNWithMeans(k=20, sim_options={
    'name': 'cosine',
    'user_based': True 
})

In [17]:
algo_KNN.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:
test_pred = algo_KNN.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9054


0.9054092791170705

Объединим два алгоритма и возьмем среднюю оценку фильма двух алгоритмов для рекомендации пользователю

In [19]:
def unseen_movies(dataset, user_id):
  movie_user = dataset[dataset['userId']==user_id]['movieId'].tolist()# id фильмов которые пользователь уже посмотрел
  movie_all = dataset['movieId'].tolist()# id всех фильмов
  movie_not_user = list(set(movie_all) - set(movie_user))# id фильмов которые пользователь не смотрел
  return movie_not_user

In [34]:
def recomendation_SVDpp(user_id):
  recommend_user_SVDpp = pd.DataFrame()
  # делаем предсказание по каждому фильму который не смотрел пользователь
  # и добавляем в датафрейм
  for i in unseen_movies(dataset, user_id):
    mov_rating = algo_SVDpp.predict(uid=user_id, iid = i).est
    row = {'movieId':i, 'ratings_SVD': mov_rating}
    recommend_user_SVDpp = pd.concat([recommend_user_SVDpp, pd.DataFrame([row])])
  return recommend_user_SVDpp.sort_values('ratings_SVD', ascending=False)

In [21]:
def recomendation_KNN(user_id):
  recommend_user_KNN = pd.DataFrame()
  # делаем предсказание по каждому фильму который не смотрел пользователь
  # и добавляем в датафрейм
  for i in unseen_movies(dataset, user_id):
    mov_rating = algo_KNN.predict(uid=user_id, iid = i).est
    row = {'movieId':i, 'ratings_KNN': mov_rating}
    recommend_user_KNN = pd.concat([recommend_user_KNN, pd.DataFrame([row])])
  return recommend_user_KNN.sort_values('ratings_KNN', ascending=False)

In [22]:
def gibrid_recomendation(user_id, n=10):
  rec_KNN = recomendation_KNN(user_id)
  rec_svd = recomendation_SVDpp(user_id)
  all_ratings = rec_svd.merge(rec_KNN, on = 'movieId')
  all_ratings['mean_ratings'] = (all_ratings['ratings_SVD']+all_ratings['ratings_KNN'])/2
  all_ratings = all_ratings.sort_values('mean_ratings', ascending=False)
  all_ratings_title = all_ratings.merge(movies[['movieId','title']], on = 'movieId' )
  return all_ratings_title['title'].tolist()[:n]

In [23]:
gibrid_recomendation(1, n=5)

['Deadfall (2012)']

Для удобства создадим класс гибридной рекомендательной системы

In [50]:
class MovieRecommender:
    def __init__(self, dataset,movies, algo_SVDpp, algo_KNN,user_id):
        self.user_id = user_id
        self.dataset = dataset
        self.movies = movies
        self.algo_SVDpp = algo_SVDpp
        self.algo_KNN = algo_KNN      

    def unseen_movies(self):
        """Возвращает список фильмов, которые пользователь еще не смотрел"""
        movie_user = self.dataset[self.dataset['userId'] == self.user_id]['movieId'].tolist()# id фильмов которые пользователь уже посмотрел
        movie_all = self.dataset['movieId'].tolist()# id всех фильмов
        movie_not_user = list(set(movie_all) - set(movie_user))# id фильмов которые пользователь не смотрел
        return movie_not_user

    def recomendation_SVDpp(self):
        """Возвращает датафрейм с фильмами и предполагаемыми оценками
           пользователя предсказанными моделью algo_SVDpp,отсортированными по убыванию"""
        recommend_user_SVDpp = pd.DataFrame()
        # делаем предсказание по каждому фильму который не смотрел пользователь
        # и добавляем в датафрейм
        for i in unseen_movies(dataset, self.user_id):
          mov_rating = algo_SVDpp.predict(uid=self.user_id, iid = i).est
          row = {'movieId':i, 'ratings_SVD': mov_rating}
          recommend_user_SVDpp = pd.concat([recommend_user_SVDpp, pd.DataFrame([row])])
        return recommend_user_SVDpp.sort_values('ratings_SVD', ascending=False)

    def recomendation_KNN(self):
        """Возвращает датафрейм с фильмами и предполагаемыми оценками
        пользователя предсказанными моделью algo_KNN,отсортированными по убыванию"""
        recommend_user_KNN = pd.DataFrame()
        # делаем предсказание по каждому фильму который не смотрел пользователь
        # и добавляем в датафрейм
        for i in unseen_movies(dataset, self.user_id):
            mov_rating = self.algo_KNN.predict(uid=self.user_id, iid=i).est
            row = {'movieId': i, 'ratings_KNN': mov_rating}
            recommend_user_KNN = pd.concat([recommend_user_KNN, pd.DataFrame([row])])
        sort_rec_KNN = recommend_user_KNN.sort_values('ratings_KNN', ascending=False)
        return sort_rec_KNN

    def hybrid_recomendation(self, n=10):
        """Возвращает n количество фильмов (по умолчанию 10) с самой высокой оценкой, расчитанной как средняя
        оценка по двум моделям algo_SVDpp и algo_KNN"""
        all_ratings = (recomendation_SVDpp(self.user_id)).merge(recomendation_KNN(self.user_id), on='movieId')
        all_ratings['mean_ratings'] = (all_ratings['ratings_SVD'] + all_ratings['ratings_KNN']) / 2
        all_ratings = all_ratings.sort_values('mean_ratings', ascending=False)
        all_ratings_title = all_ratings.merge(self.movies[['movieId', 'title']], on='movieId')
        return all_ratings_title['title'].tolist()[:n]


In [52]:
rec = MovieRecommender(dataset, movies, algo_SVDpp, algo_KNN, 1)

In [53]:
rec.hybrid_recomendation()

["Guess Who's Coming to Dinner (1967)",
 'Touch of Evil (1958)',
 'Creature Comforts (1989)',
 'Paths of Glory (1957)',
 'His Girl Friday (1940)',
 "Rosemary's Baby (1968)",
 'Unforgiven (1992)',
 'Lawrence of Arabia (1962)',
 'Rebecca (1940)',
 'Streetcar Named Desire, A (1951)']